In [1]:
#Imports
import numpy as np
from load_hyperparams import load_hyperparams
from predict_satellites import get_satellite_properties
from predict_orphans import get_orphan_satellite_properties
from fix_lmc_coords import get_lmc_coords, rotate_about_LMC
from astropy.coordinates import SkyCoord
from masks import load_mask, evaluate_mask
from ssf import load_ssf, apply_ssfs

In [2]:
#Set parameters
params = {}

params['alpha'] = -1.44
params['sigma_M'] = 0.2
params['M50'] = 7.5
params['sigma_mpeak'] = 0.1
params['B'] = 1.
params['A'] = 30
params['sigma_r'] = 0.3
params['n'] = 1.

#Load hyperparameters
hparams, cosmo_params, orphan_params, halo_data, vpeak_Mr_interp = load_hyperparams()

In [3]:
#Load survey masks and ssfs
surveys = ['des', 'ps1']

masks = {}
ssfs = {}

print('loading masks and ssfs ...')
for survey in surveys:
    masks[survey] = load_mask('{}'.format(survey))
    ssfs[survey] = load_ssf(survey)
print('done')

loading masks and ssfs ...
NSIDE = 4096
ORDERING = NESTED in fits file
INDXSCHM = IMPLICIT
NSIDE = 128
ORDERING = RING in fits file
INDXSCHM = IMPLICIT


/Users/eon/Documents/subhalo_satellite_connection/1912.03303/ssf.py:12: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  self.config = yaml.load(open(config_file))


NSIDE = 4096
ORDERING = NESTED in fits file
INDXSCHM = IMPLICIT
NSIDE = 128
ORDERING = RING in fits file
INDXSCHM = IMPLICIT
done


In [4]:
#Return satellite properties for a particular MW-like host halo
satellite_properties = get_satellite_properties(halo_data[14], params, hparams, cosmo_params, vpeak_Mr_interp)
satellite_properties

{'Mr': array([-1.75110097e+01, -1.72718366e+01, -1.17511860e+01, -1.24179238e+01,
        -1.39490887e+01, -1.43405290e+01, -8.77561620e+00, -1.04344806e+01,
        -9.55438070e+00, -1.04772788e+01, -5.74891093e+00, -7.83812762e+00,
        -7.36871288e+00, -7.97053260e+00, -1.24631551e+01, -9.08237692e+00,
        -7.32601964e+00, -9.21457329e+00, -6.26746159e+00, -9.12759015e+00,
        -6.99916485e+00, -1.02996942e+01, -6.74382689e+00, -6.72631121e+00,
        -7.73195838e+00, -5.18877240e+00, -6.91655554e+00, -4.09845643e+00,
        -6.17242960e+00, -6.71076849e+00, -4.03879536e+00, -6.84597950e+00,
        -5.45775805e+00, -5.19143973e+00, -6.49180765e+00, -4.78211901e+00,
        -9.91114678e+00, -1.06915428e+01, -9.81425776e+00, -4.49780214e+00,
        -4.82321522e+00, -3.47107698e+00, -7.18691647e+00, -4.86182335e+00,
        -6.73652474e+00, -5.68636134e+00, -6.85725325e+00, -4.72259815e+00,
        -8.46770630e+00, -3.53218332e+00, -2.46555633e+00, -5.55278226e+00,
      

In [5]:
#Return orphan satellite properties for a particular host halo
orphan_satellite_properties = get_orphan_satellite_properties(halo_data[14], params, hparams, 
                                                              cosmo_params, vpeak_Mr_interp)
print(orphan_satellite_properties)

{'Mr': array([-3.25300538e+00, -5.51506878e+00, -7.10467706e+00, -7.81859433e+00,
       -1.57469603e+00,  1.77601463e+00, -2.38583945e+00, -1.02012880e+00,
        7.40568114e-01,  1.32741681e+00, -7.51308412e-01,  1.46820760e+00,
        1.32567190e+00,  1.91747339e+00,  1.50288854e+00,  1.21385040e+00,
        1.50485571e+00,  1.16701063e+00, -1.38200939e+00,  8.11942702e-01,
        1.55256261e+00,  1.62866976e+00, -1.75848917e+00,  1.23706205e+00,
        2.66619315e-01,  7.78938242e-01,  4.39863040e-01,  1.48479117e+00,
        1.93147084e+00,  1.40044346e+00,  2.43011612e+00,  7.95269681e-01,
        1.39558001e+00,  1.82998681e+00, -5.14120812e-01,  9.43364408e-01,
        8.52045299e-02,  1.14394595e+00,  2.22699380e+00,  2.03708836e+00,
        2.10943258e+00, -2.36144610e+00,  1.81388806e+00,  5.09843558e-01,
        5.68701882e-01,  1.71252699e+00,  1.61338181e+00, -1.93782927e+00,
        1.16458234e+00, -1.38010898e+00,  6.96696156e-01,  8.15168523e-01,
        2.40453110

In [6]:
#Rotate about LMC
lmc_ind = 0
lmc_cartesian_coords = get_lmc_coords(halo_data[14],cosmo_params,lmc_ind)
satellite_properties_rotated = rotate_about_LMC(satellite_properties,halo_data[0],cosmo_params,
                                                lmc_cartesian_coords,lmc_ind,observer_ind=0)

In [7]:
#Precompute satellite properties and flags for each observer location
satellite_properties_list = []
DES_flags_list = []
PS1_flags_list = []
Halo_ra_list = []
Halo_dec_list = []

#Loop over simulations
for ind in [14, 20]:
    #Loop over observer locations
    for i in range(6):
        #Get satellite properties
        satellite_properties = get_satellite_properties(halo_data[ind], params, hparams, cosmo_params, vpeak_Mr_interp)
        satellite_properties_rotated = rotate_about_LMC(satellite_properties,halo_data[ind],cosmo_params,
                                                    lmc_cartesian_coords,lmc_ind,i)
        #Transform to sky coordinates
        Halo_sky_coord = SkyCoord(x=satellite_properties_rotated['rotated_pos'][:,0], 
                          y=satellite_properties_rotated['rotated_pos'][:,1], 
                          z=satellite_properties_rotated['rotated_pos'][:,2], 
                          unit='kpc', representation_type='cartesian').spherical
        satellite_properties_rotated['ra'] = Halo_sky_coord.lon.degree
        satellite_properties_rotated['dec'] = Halo_sky_coord.lat.degree
        #Assign flags 
        for survey in surveys:
            satellite_properties_rotated['{}_flags'.format(survey)] = evaluate_mask(Halo_sky_coord.lon.degree,
                                                                                    Halo_sky_coord.lat.degree,
                                                                                    masks[survey], survey)
        #Apply ssfs
        satellite_properties_rotated['pdet'] = apply_ssfs(satellite_properties_rotated,ssfs)
        #Append
        satellite_properties_list.append(satellite_properties_rotated)

In [8]:
#First look at predictions
des_sum = []
ps1_sum = []

for i in range(len(satellite_properties_list)):
    des_flags = satellite_properties_list[i]['des_flags']
    ps1_flags = np.logical_and(satellite_properties_list[i]['ps1_flags'],~satellite_properties_list[i]['des_flags'])
    des_sum.append(np.sum(satellite_properties_list[i]['pdet'][des_flags]))
    ps1_sum.append(np.sum(satellite_properties_list[i]['pdet'][ps1_flags]))
    
print('number of predicted des satellites = {}'.format(np.mean(des_sum,axis=0)))
print('number of predicted ps1 satellites = {}'.format(np.mean(ps1_sum,axis=0)))

number of predicted des satellites = 18.085088292035866
number of predicted ps1 satellites = 24.70953608952072
